In [93]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OrdinalEncoder

## To do:
Convert dates to Unix
Convert object columns to categorical
set up XGBoost


In [47]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [48]:
train['Category'].unique()

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING',
       'SEX OFFENSES NON FORCIBLE', 'EXTORTION', 'GAMBLING', 'BAD CHECKS',
       'TREA', 'RECOVERED VEHICLE', 'PORNOGRAPHY/OBSCENE MAT'],
      dtype=object)

In [49]:
train

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607


In [50]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


In [51]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884262 entries, 0 to 884261
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Id          884262 non-null  int64  
 1   Dates       884262 non-null  object 
 2   DayOfWeek   884262 non-null  object 
 3   PdDistrict  884262 non-null  object 
 4   Address     884262 non-null  object 
 5   X           884262 non-null  float64
 6   Y           884262 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 47.2+ MB


In [52]:
train_drop_list = ['Descript', 'Resolution', 'Address', 'Dates']
# test_drop_list = ['Address', 'Id', 'Dates']
object_list = ['DayOfWeek', 'PdDistrict', 'Address', 'Category', 'Dates']


train['Dates'] = pd.to_datetime(train['Dates'], infer_datetime_format = True)
test['Dates'] = pd.to_datetime(train['Dates'], infer_datetime_format = True)

print(train['Dates'])

train['TimeStamp'] = train['Dates'].values.astype(np.int64) // 10 ** 9
# test['TimeStamp'] = test['Dates'].values.astype(np.int64) // 10 ** 9
train = train.drop(train_drop_list, axis = 1)
# test = test.drop(test_drop_list, axis = 1)
    
for column_name in object_list:
    try:
        train[column_name] = train[column_name].astype('category')
    except:
        continue
    try:
        test[column_name] = test[column_name].astype('category')
    except:
        continue


train.info()

0        2015-05-13 23:53:00
1        2015-05-13 23:53:00
2        2015-05-13 23:33:00
3        2015-05-13 23:30:00
4        2015-05-13 23:30:00
                 ...        
878044   2003-01-06 00:15:00
878045   2003-01-06 00:01:00
878046   2003-01-06 00:01:00
878047   2003-01-06 00:01:00
878048   2003-01-06 00:01:00
Name: Dates, Length: 878049, dtype: datetime64[ns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   Category    878049 non-null  category
 1   DayOfWeek   878049 non-null  category
 2   PdDistrict  878049 non-null  category
 3   X           878049 non-null  float64 
 4   Y           878049 non-null  float64 
 5   TimeStamp   878049 non-null  int64   
dtypes: category(3), float64(2), int64(1)
memory usage: 22.6 MB


In [75]:
# weekdays = train['DayOfWeek']
OrdEncoder = OrdinalEncoder()
train[['PdDistrict']] = OrdEncoder.fit_transform(train[['PdDistrict']])
# test[['PdDistrict']] = OrdEncoder.fit_transform(test[['PdDistrict']])

train[['Category']] = OrdEncoder.fit_transform(train[['Category']])

In [76]:
week_day_dict = {
    'Sunday':0,
    'Monday':1,
    'Tuesday' : 2,
    'Wednesday' : 3,
    'Thursday' : 4,
    'Friday' : 5,
    'Saturday' :6
}

train['DayOfWeek'] = train['DayOfWeek'].replace(week_day_dict)
# test['DayOfWeek'] = test['DayOfWeek'].replace(week_day_dict)

train['DayOfWeek'] = train['DayOfWeek'].astype('int64')
# test['DayOfWeek'] = test['DayOfWeek'].astype('int64')

In [77]:
train

,Category,DayOfWeek,PdDistrict,X,Y,TimeStamp
0,37.0,3,4.0,-122.425892,37.774599,1431561180
1,21.0,3,4.0,-122.425892,37.774599,1431561180
2,21.0,3,4.0,-122.424363,37.800414,1431559980
3,16.0,3,4.0,-122.426995,37.800873,1431559800
4,16.0,3,5.0,-122.438738,37.771541,1431559800
...,...,...,...,...,...,...
878044,25.0,1,8.0,-122.459033,37.714056,1041812100
878045,16.0,1,2.0,-122.447364,37.731948,1041811260
878046,16.0,1,7.0,-122.403390,37.780266,1041811260
878047,35.0,1,7.0,-122.390531,37.780607,1041811260


In [78]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Category    878049 non-null  float64
 1   DayOfWeek   878049 non-null  int64  
 2   PdDistrict  878049 non-null  float64
 3   X           878049 non-null  float64
 4   Y           878049 non-null  float64
 5   TimeStamp   878049 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 40.2 MB


In [79]:
print(train.shape)

(878049, 6)


In [80]:
train_target = train['Category']
train_variables = train.drop('Category', axis = 1)
train_variables

,DayOfWeek,PdDistrict,X,Y,TimeStamp
0,3,4.0,-122.425892,37.774599,1431561180
1,3,4.0,-122.425892,37.774599,1431561180
2,3,4.0,-122.424363,37.800414,1431559980
3,3,4.0,-122.426995,37.800873,1431559800
4,3,5.0,-122.438738,37.771541,1431559800
...,...,...,...,...,...
878044,1,8.0,-122.459033,37.714056,1041812100
878045,1,2.0,-122.447364,37.731948,1041811260
878046,1,7.0,-122.403390,37.780266,1041811260
878047,1,7.0,-122.390531,37.780607,1041811260


The train dataset will be divided into further training and testing sets. 

In [81]:
X_train, X_test, y_train, y_test = train_test_split(train_variables, train_target, test_size = 0.25, random_state = 42)

In [84]:
X_train_DMatrix = xgb.DMatrix(X_train, label = y_train)
X_test_DMatrix = xgb.DMatrix(X_test, label = y_test)

In [86]:
params = {
    'objective': 'multi:softmax',
    'num_class': len(train_target.unique()),
    'max_depth': 6,
    'eta': 0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

num_rounds = 100
xgb_model = xgb.train(params, X_train_DMatrix, num_rounds)

In [88]:
y_pred = xgb_model.predict(X_test_DMatrix)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.2811131914738535


In [94]:
cm = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{cm}')
cr = classification_report(y_test, y_pred)
print(f'Classification Report:\n{cr}')

Confusion Matrix:
[[   1   34    0 ...   29    1    0]
 [   0 1403    0 ... 1000   25    2]
 [   0    0    0 ...    8    0    0]
 ...
 [   0  335    0 ... 3034    4    1]
 [   1  350    0 ...  310   85    3]
 [   0  192    0 ...  111    5    8]]


y:\Anaconda\envs\LHLenvironment\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
y:\Anaconda\envs\LHLenvironment\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
              precision    recall  f1-score   support

         0.0       0.07      0.00      0.01       382
         1.0       0.20      0.07      0.11     19231
         2.0       0.00      0.00      0.00        89
         3.0       0.00      0.00      0.00        72
         4.0       0.18      0.01      0.01      9247
         5.0       0.21      0.03      0.05      1018
         6.0       0.00      0.00      0.00       585
         7.0       0.33      0.43      0.37     13457
         8.0       0.15      0.00      0.01      1076
         9.0       0.00      0.00      0.00       274
        10.0       0.00      0.00      0.00        58
        11.0       0.08      0.01      0.02       106
        12.0       0.25      0.03      0.05      2604
        13.0       0.37      0.01      0.01      4036
        14.0       0.00      0.00      0.00        43
        15.0       0.00      0.00      0.00       549
        16.0       0.30      0.75      0.43     43883
    

y:\Anaconda\envs\LHLenvironment\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
